This script contains three functions:
1. clean_set
    This function opens an excel file and deletes Unnamed columns created when seperating excel to different sheets, iterates through
    the numeric columns to find and replace non-numeric values with np.NaN, updates the types for each column, and saves the df as a 
    csv ending in '_clean'.

2. get_countries_not_included (from Daniel)
    Iterates through user provided country list and compares it to the json dictionary 'iso_country_dic'. The function produces a 
    list of countries not included in the dictionary for user to add manually.
 
 3. pull_country_not_included
    Creates country list from user provided local path and feeds it through the get_countries_not_included function to pull list for user to add manually (as stated
    above)
 
 
The following data sets were cleaned with this script:

- 2017_salary
- ann_instruction_hrs
- expenditure_per_stud
- GDPpercap_PPPadj
- govt_regulations
- PISA_scores
- salaryRatio_by_exp
- school_age_duration
- school_types
- support_staff
- teaching_time

In [13]:

import pandas as pd 
import numpy as np 
import json

def clean_set(relative_path):

    ### read in our files
    with open(r'C:\Users\diego\Desktop\Project Folder\data-cleaning\iso_country_codes\iso_country_codes.json') as fp:
        iso_country_dic = dict( json.load(fp))

    xl = pd.read_excel(relative_path)

    df = xl.copy()

    ### list our column names and loop thru to add true columns to keep list to re-save later
    columns = list(df.columns)
    
    keep = []

    for item in columns:
        if item.startswith('Unnamed'):
            pass
        else:
            keep.append(item)

    df = df.loc[:, df.columns.intersection(keep)]

    ### get numeric columns and loop thru to append non-numeric values to replace list (to replace w/ NaN)
    num_columns = list(df.columns[1:])

    replace = []

    for line in num_columns:
        #df[line] = df[line].map(dict(Yes = 1, No = 0))             #unhash to change yes/no values to binary
        unique = list(df[line].unique())
        for item in unique:
            if isinstance(item,str) == True :
                if item in replace:
                    pass
                else:
                    replace.append(item)

    df = df.replace(replace,np.nan)


    ### update column types and add ID column 
    df[df.columns[0]] = df[df.columns[0]].astype('category')

    df[num_columns] = df[num_columns].astype('float')

    df['ID']=df.iloc[:, 0].str.lower().map(iso_country_dic).astype('category')

    ### save with _clean as csv file
    relative_path = relative_path.replace('.xls','_clean.csv')

    df.to_csv(relative_path, index=False)

#clean_set(input())

In [62]:
def get_countries_not_included(countries_list):
    
    #change to local path
    with open(r'C:\Users\diego\Desktop\Project Folder\data-cleaning\iso_country_codes\iso_country_codes.json') as fp: 

        iso_country_dic = dict( json.load(fp))
    
    not_included_countries = []
    for c in countries_list:
        if type(c) != str:
            pass
        elif (c.lower() not in iso_country_dic):
            not_included_countries.append(c)

    return not_included_countries

In [86]:
def pull_country_not_incl(path):

    csv = pd.read_csv(path)         #read in file

    df = csv.copy()

    country_list = list(df.iloc[:, 0]) #create country list by calling column 0

    not_included_list = get_countries_not_included(country_list)

    return not_included_list

#pull_country_not_incl(input())